In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
import torch
import torch.nn as nn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# load data
train= pd.read_csv('../input/leaf-classification/train.csv.zip')
test = pd.read_csv('../input/leaf-classification/test.csv.zip')

In [ ]:
def encode(train, test):
    label_encoder = LabelEncoder().fit(train.species)
    labels = label_encoder.transform(train.species)
    classes = list(label_encoder.classes_)
    
    train = train.drop(['species', 'id'], axis=1)
    index = test["id"]
    test = test.drop('id', axis=1)
    test_data = test.copy()
    columns=test.columns

    return train, labels, test, classes,test_data,columns

train, labels, test, classes ,test_data,columns= encode(train, test)

# standardize train and test features

scaler = StandardScaler().fit(train.values)
data = scaler.transform(train.values)
test= scaler.transform(test.values)





In [ ]:
test_ratio = 0.2
x_train, x_valid, y_train, y_valid = train_test_split(data, labels, test_size=test_ratio, random_state=1, shuffle=True)

In [ ]:
x_train = torch.tensor(x_train).float()
x_test = torch.tensor(test).float()
x_valid = torch.tensor(x_valid).float()

y_train = torch.tensor(y_train).float()
y_valid = torch.tensor(y_valid).float()


num_hidden1 = 256
num_features = x_train.shape[1]
num_classes =100

model = nn.Sequential(nn.Linear(num_features, num_hidden1),
                    nn.ReLU(),
                    nn.Linear(num_hidden1, num_classes),
                    nn.Softmax(),
                    )
# Loss
loss = nn.CrossEntropyLoss()
# Optimization
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train
num_epochs = 500
num_samples_train = x_train.shape[0]
num_samples_valid = x_valid.shape[0]



for epoch in range(num_epochs):
    optimizer.zero_grad()
    yp = model(x_train)
    y_train = torch.tensor(y_train).long()
    loss_value = loss(yp, y_train)
    yp = torch.argmax(yp, dim=1)
        
    num_corrects = torch.sum(yp == y_train)
    acc_train = num_corrects.float() / float(num_samples_train)
    loss_value.backward()
    optimizer.step()

    yp = model(x_valid)
    yp = torch.argmax(yp, dim=1)
    num_corrects = torch.sum(yp == y_valid)
 
    acc_valid = num_corrects.float() / float(num_samples_valid)
    print('Epoch: ', epoch, ', Train Loss: ', loss_value.item(),
          ', Train Accuracy: ', acc_train.item(),
          ', Validation Accuracy: ', acc_valid.item())


yp = model(x_test)
yp = torch.max(yp, dim=1)


In [ ]:
labels_test=[classes[i]for i in (yp.indices)]
prop_test=[i.item() for i in (yp.values)]

In [ ]:
scor=pd.DataFrame(prop_test,columns=["score"])

In [ ]:
species = pd.DataFrame(labels_test,columns=["species"])

In [ ]:
submission = pd.concat([scor, species], axis=1)
submission.to_csv('submission.csv', index = False)

In [ ]:
submission